In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from pyjarowinkler import distance
import faiss

In [2]:
def get_model_tokenizer(hub_name: str):
    tokenizer = AutoTokenizer.from_pretrained(hub_name)
    model = AutoModel.from_pretrained(hub_name)
    return model, tokenizer

class BertCLS(nn.Module):
    def __init__(self, model, n_classes):
        super(BertCLS, self).__init__()
        self.model = model
        self.fc = nn.Linear(312, n_classes)
    
    def forward(self, batch):
        return self.fc(self.model(**batch).pooler_output)
    
def get_embeddings(bert_cls, tokenizer, text):
    tokens = tokenizer(text, padding=True,
                       max_length=300, truncation=True,
                       return_tensors='pt')
    tokens = tokens.to(bert_cls.model.device)
    return bert_cls.model(**tokens).pooler_output.detach().cpu().numpy()

def string_dist(str1, str2):
    return distance.get_jaro_distance(str1, str2,
                                      winkler=True,
                                      winkler_ajustment=True,
                                      scaling=0.2)

In [3]:
model, tokenizer = get_model_tokenizer("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

In [5]:
bert_cls = BertCLS(model, 1005)
bert_cls.load_state_dict(torch.load("./BertCLS_epoch_3_1000.pth", map_location=torch.device("cpu")))
bert_cls = bert_cls.to(device)

In [6]:
final_df = pd.read_feather("bert-tiny-1000-final-df.feather")
search_df = final_df[final_df.columns[312:]].reset_index(drop=True)

In [7]:
embeddings = final_df[final_df.columns[:312]].values.astype(np.float32)
embeddings = np.ascontiguousarray(embeddings)
faiss.normalize_L2(embeddings)

In [8]:
d = 312 # длина эмбеддинга
index = faiss.IndexFlatIP(d)
index.add(embeddings) # сами эмбеддинги, нампай массив shape = (n_samples, d)

In [9]:
text_query = "стул деревянный"
xq = get_embeddings(bert_cls, tokenizer, text_query)
faiss.normalize_L2(xq)

In [10]:
k = 100
D, I = index.search(xq, k) # xq shape = (1, d)

In [11]:
faiss_results = search_df.iloc[I[0]].reset_index(drop=True)
faiss_results["string_dist"] = faiss_results["product_name"].apply(lambda x: string_dist(x, text_query))
faiss_results

,product_name,price,product_vat_rate,product_msr,product_characteristics,okpd2_code,okpd2_name,inn,country_code,okpd2_value,text,string_dist
0,стул ученический,2000.00,20%,Штука,,32.99.59.000,"Изделия различные прочие, не включенные в друг...",8764d748838ad7fc3a975f3757167aec,643,32,стул ученический [SEP],0.94
1,стул ученический лабораторный,4251.57,Без НДС,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",247ca9307cf09e6b9e3bb8b1c9212e83,643,31,стул ученический лабораторный [SEP],0.92
2,стул изо хром,2000.00,0%,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",86875838b24b114a9289fc313e90788d,643,31,стул изо хром [SEP],0.92
3,стул,2055.00,Без НДС,Штука,,32.99.59.000,"Изделия различные прочие, не включенные в друг...",4bdab417f1773e8e76887a99c27cde5a,643,32,стул [SEP],0.95
4,стул тип 2,9740.89,0%,Штука,,31.09.99.211,Услуги по изготовлению мягкой мебели по индиви...,e3674a861c1e906db077407701580cf2,643,31,стул тип 2 [SEP],0.92
...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Стул ученический, регулировка высоты (4-6 гр.р.)",1333.75,Без НДС,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",ba7d6516f94e3fae34f89701d8f11fde,643,31,"Стул ученический, регулировка высоты (4-6 гр.р...",0.56
96,стол манипуляционный,9441.89,Без НДС,Штука,,32.50.21.111,Инструменты терапевтические,6bdb620dfd87a464cf19992889a98a97,643,32,стол манипуляционный [SEP],0.79
97,Стул нерегулируемый,3188.00,Без НДС,Штука,,31.09.12.121,Кровати деревянные для взрослых,887701edda2e879deed0ed618185b9d3,643,31,Стул нерегулируемый [SEP],0.71
98,кресло тип 3,15946.66,Без НДС,Штука,,31.01.12.160,"Мебель для сидения, преимущественно с деревянн...",441fbd989720975cf5d6b0a7a098badc,643,31,кресло тип 3 [SEP],0.47


In [12]:
faiss_results.sort_values(by="string_dist", ascending=False).head(10)

,product_name,price,product_vat_rate,product_msr,product_characteristics,okpd2_code,okpd2_name,inn,country_code,okpd2_value,text,string_dist
8,стул детский,2500.00,20%,Штука,,31.09.13.190,"Мебель деревянная прочая, не включенная в друг...",260784b5cc1c089d1409e90ddefa8a3d,643,31,стул детский [SEP],0.95
6,стул офисный,10700.00,Без НДС,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",165dc33671d7778cb9dd50238a7d7e5e,643,31,стул офисный [SEP],0.95
3,стул,2055.00,Без НДС,Штука,,32.99.59.000,"Изделия различные прочие, не включенные в друг...",4bdab417f1773e8e76887a99c27cde5a,643,32,стул [SEP],0.95
84,стул на деревянном каркасе,21505.00,20%,Штука,,31.01.12.160,"Мебель для сидения, преимущественно с деревянн...",72d10f8a4fcf25bcfa9159c867a02a7d,643,31,стул на деревянном каркасе [SEP],0.94
11,стул медицинский,8018.00,0%,Штука,,32.50.30.119,"Мебель медицинская прочая, включая хирургическ...",926dc3e32feb99196cb926959af17dbd,643,32,стул медицинский [SEP],0.94
0,стул ученический,2000.00,20%,Штука,,32.99.59.000,"Изделия различные прочие, не включенные в друг...",8764d748838ad7fc3a975f3757167aec,643,32,стул ученический [SEP],0.94
10,стул ученический (тип2),3533.33,Без НДС,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",f0ab568683992c6d0724646692ed9ccd,643,31,стул ученический (тип2) [SEP],0.93
13,стул ученический (тип6),2773.33,Без НДС,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",f0ab568683992c6d0724646692ed9ccd,643,31,стул ученический (тип6) [SEP],0.93
5,стул тип 1,22821.76,0%,Штука,,31.09.99.211,Услуги по изготовлению мягкой мебели по индиви...,e3674a861c1e906db077407701580cf2,643,31,стул тип 1 [SEP],0.92
1,стул ученический лабораторный,4251.57,Без НДС,Штука,,31.01.11.150,"Мебель для сидения, преимущественно с металлич...",247ca9307cf09e6b9e3bb8b1c9212e83,643,31,стул ученический лабораторный [SEP],0.92
